# key for theyworkforyou API
DN8s9LBm8jMBFZihXEG2gqzx

In [1]:
APIkey = 'DN8s9LBm8jMBFZihXEG2gqzx'

In [2]:
import requests

In [3]:
def getQuota():
    quotaurl = 'https://www.theyworkforyou.com/api/getQuota'
    response = requests.get(quotaurl, params={'key':APIkey})
    return response.json()['quota']

In [4]:
getQuota()

{'limit': 1000, 'current': 0}

In [6]:
postcodes_uk.validate('GL205FL')

True

In [31]:
import requests
import postcodes_uk

def is_valid_postcode(postcode):
    """
    This function takes one argument and returns True or False.
    
    Args:
    postcode (str): The postcode
    
    Returns:
    bool: True if a valid postcode, otherwise returns False.
    """
    #making sure the postcode is in uppercase, required for validation
    postcode = postcode.upper()
    # using the postcodes_uk library to validate the postcode
    ret = postcodes_uk.validate(postcode)
    return ret

def get_MP_info(postcode_or_constituency):
    """
    This function takes one argument, either the postcode or the constituency name, and returns the MP information from Hansard.
    
    Args:
    postcode_or_constituency (str): Postcode or a Constituency
    
    Returns:
    dict: The MP information from Hansard
    """
    # checks to see in the input is a valid postcode
    params = {'key':APIkey}
    if is_valid_postcode(postcode_or_constituency):
        params['postcode'] = postcode_or_constituency
    else:
        params['constituency'] = postcode_or_constituency
    
    url = 'https://www.theyworkforyou.com/api/getMP'
    response = requests.get(url, params=params)
    return response.json()

In [32]:
MP_info = get_MP_info('Tewkesbury')

In [33]:
MP_info

{'member_id': '42617',
 'house': '1',
 'constituency': 'Tewkesbury',
 'party': 'Conservative',
 'entered_house': '2019-12-13',
 'left_house': '9999-12-31',
 'entered_reason': 'general_election',
 'left_reason': 'still_in_office',
 'person_id': '10505',
 'lastupdate': '2019-12-13 05:21:12',
 'title': '',
 'given_name': 'Laurence',
 'family_name': 'Robertson',
 'full_name': 'Laurence Robertson',
 'url': '/mp/10505/laurence_robertson/tewkesbury',
 'image': '/people-images/mpsL/10505.jpg',
 'image_height': 160,
 'image_width': 120,
 'office': [{'moffice_id': 'uk.parliament.data/Member/253/Committee/11/1',
   'dept': 'Panel of Chairs',
   'position': 'Member',
   'from_date': '2020-01-15',
   'to_date': '9999-12-31',
   'person': '10505',
   'source': ''}]}

In [68]:
personID = MP_info['person_id']
params = {'key':APIkey}
params['person'] = personID
#params['num'] = "100"
url = 'https://www.theyworkforyou.com/api/getHansard'
response = requests.get(url, params=params)
response

<Response [200]>

In [56]:
personID

'10505'

In [73]:
def get_Hansard(person_id):
    """
    This function takes one argument and returns the MP's person ID from Hansard.
    
    Args:
    personID (int): gets the relevant data from Hansard
    
    Returns:
    dict: The information from Hansard about the person_id
    """
    params = {'key':APIkey}
    params['person'] = person_id
    url = 'https://www.theyworkforyou.com/api/getHansard'
    response = requests.get(url, params=params)
    data = response.json()
    params['page'] = 1
    # loops until we have all the pages
    while True:
        params['page'] += 1
        response = requests.get(url, params=params)
        data0 = response.json()
        if len(data0['rows']) == 0:
            break
        data['rows'].extend(data0['rows'])
    return data

In [75]:
data = get_Hansard(MP_info['person_id'])

In [85]:
import numpy as np
np.argmax([len(x['body']) for x in data['rows'][:128]])

54

In [86]:
data['rows'][54]

{'gid': '2022-11-16a.319.2',
 'hdate': '2022-11-16',
 'htime': '16:36:00',
 'section_id': '27686696',
 'subsection_id': '27686696',
 'htype': '12',
 'major': '2',
 'minor': '0',
 'person_id': '10505',
 'hpos': '90',
 'video_status': '0',
 'epobject_id': '27686700',
 'body': '<p pid="a319.2/1">It is a pleasure to serve under your chairmanship, Sir Gary.  It is also a pleasure to see my right hon. Friend <phrase class="honfriend" name="Andrew Mitchell" person_id="uk.org.publicwhip/person/11115">the  Member for Sutton Coldfield (Mr Mitchell)</phrase> back in the Government. Like me, he greatly values the effect that British aid has had over very many years and wants it to continue. I know he takes a deep interest in these subjects, as does my hon. Friend <phrase class="honfriend" name="James Duddridge" person_id="uk.org.publicwhip/person/11785">the Member for Rochford and Southend East (Sir James Duddridge)</phrase>&#8212;it is really good to see him in the debate.</p><p pid="a319.2/2">We

In [88]:


from bs4 import BeautifulSoup
BeautifulSoup(data['rows'][54]['body'], "html.parser").text



'It is a pleasure to serve under your chairmanship, Sir Gary.  It is also a pleasure to see my right hon. Friend the  Member for Sutton Coldfield (Mr Mitchell) back in the Government. Like me, he greatly values the effect that British aid has had over very many years and wants it to continue. I know he takes a deep interest in these subjects, as does my hon. Friend the Member for Rochford and Southend East (Sir James Duddridge)—it is really good to see him in the debate.We have held a number of debates on Ethiopia. I have secured an urgent question and have taken part in the debates. I have chaired the all-party parliamentary group on Ethiopia for a dozen years or so, and I continue to take a deep interest in the country. I am very sorry to see what has been happening over the past few years. I was at the Ethiopian embassy two weeks ago. The ceasefire had been announced the night before; it was a very moving moment and there was a lot of hope. I am very hopeful that we can make progres

In [ ]:
from bs4 import BeautifulSoup
import string
extracts = [BeautifulSoup(x['body'], 'html.parser').text.lower().translate(str.maketrans('', '', string.punctuation)) for x in data['rows']]

In [ ]:
extracts[0]


In [ ]:
extracts = [e for e in extracts if len(e) > 512]

In [ ]:
100.0 * len([x for x in extracts if 'horse' in x]) / len(extracts)

In [ ]:
from wordcloud import WordCloud

In [ ]:
import matplotlib.pyplot as plt

# lower max_font_size
wc = WordCloud(max_font_size=40, background_color="black")
wc.stopwords = wc.stopwords.union({'speaker', 'secretary',
                                   'state', 'statement',
                                  'will', 'make', 'available'
                                   'made', 'ask', 'steps', 'b',
                                  'assessment', 'made', 'whether',
                                  'ensure', 'taking', 'take',
                                  'use', 'member', 'minister', 'friend', 'members', 'hon', 'mr','gentleman', 'lady', 'u', ';'})
wordcloud = wc.generate('\n'.join(extracts).lower())
plt.figure( figsize=(25,15) )
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")


In [ ]:
from rake_nltk import Rake
rake_nltk_var = Rake()
keywords = []
for e in extracts:
    rake_nltk_var.extract_keywords_from_text(e)
    keywords.append(rake_nltk_var.get_ranked_phrases()[:8])

In [ ]:
# lower max_font_size
wc = WordCloud(max_font_size=40, background_color="black")
wc.stopwords = wc.stopwords.union({'speaker', 'secretary',
                                   'state', 'statement',
                                  'will', 'make', 'available'
                                   'made', 'ask', 'steps', 'b',
                                  'assessment', 'made', 'whether',
                                  'ensure', 'taking', 'take',
                                  'use', 'member', 'minister', 'friend',
                                'members', 'hon', 'mr','gentleman',
                                 'lady', 'u', ';',
                                 'people', 'many'})
wordcloud = wc.generate(' '.join([' '.join(k) for k in keywords]))
plt.figure( figsize=(25,15) )
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
rake_nltk_var.get_ranked_phrases()[:8]

In [ ]:
K = ' '.join([' '.join(k) for k in keywords])

In [ ]:
K[:128]

In [ ]:
K.count(' ')